# AWS and Intel Hackathon for Good: Notebook para Data Augmentation

Arturo Pinar

Alejandro Pinar

Alberto Jiménez


El objetivo de este notebook es el de aumentar el número de imágenes de cada clase en el dataset. El principal motivo para este aumento se debe a el equilibrado del dataset. 

Cuando un dataset presenta un número de imágenes muy distinto en cada una de sus clases decimos que es un dataset no equilibrado y puede llevar a que el modelo no se entrene correctamente y no aprenda a generalizar (overfitting). En el caso del dataset utilizado para esta competición el número de imagenes por clase es el siguiente: 

- Clase 1 = 89 imágenes para test. 
- Clase 2  = 50 imágenes de test. 
- Clase 3 = 13 imágenes de test. 
- Clase 4 = 62 imágenes de test.
- Clase 5 = 100 imágenes de test. 

Como las diferencias son muy grandes entre las distintas clases, además de que para la clase 3 el número de imágenes el claramente insuficiente, este dataset necesita ser aumentado para evitar overfitting en el modelo. 

Ejecutando cada una de las celdas en orden de este notebook se puede realizar el proceso de augmentation utilizando las transformaciones ofrecidas por Pytorch. 


# 1. Carga de librerías necesarias

## 1.1. Dependencias

Será necesario abrir una shell con Anaconda Prompt y lanzar el siguiente comando: 

``conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch``

## 1.2. Librerías

In [1]:
import numpy as np
import pandas as pd
import random 

from IPython.display import display

from tqdm.notebook import tqdm

import torch
import torchvision

from torchvision import transforms
import os
import glob
from PIL import Image

C:\Users\apina\anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\apina\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
root_path_training = "train/"

## 1.3. Configuración del número máximo de imágenes por clase deseado

Para cambiar el numero de imagenes basta con cambiar el valor de la variable "max_images"

In [26]:
max_images = 1000

# 2. Data Loading and Preprocessing
## 2.1. Data Augmentation

In [33]:
n_images_class = []

n_images = 0

for i in range(1,6):
    class_path = root_path_training + str(i) + "/"
    img_list = os.listdir(class_path)
    
    for j in (img_list):
      if j[-4:]=='.jpg':
        n_images = n_images + 1
    n_images_class.append(n_images)
    print("class ", i, "number of images: ", str(n_images))
    n_images = 0

class  1 number of images:  1000
class  2 number of images:  1000
class  3 number of images:  1000
class  4 number of images:  1000
class  5 number of images:  1000


In [34]:
n_images_class

[1000, 1000, 1000, 1000, 1000]

In [30]:
transform = {
    'RandomResizedCrop':
    transforms.Compose([
          transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
          transforms.Resize(size=256),
          #transforms.ToTensor(),
    ]),
    'RandomCrop':
    transforms.Compose([
        transforms.RandomCrop(120, padding = 20),
        transforms.Resize(size=256),
        #transforms.ToTensor(),
    ]),
    'RandomRotation':
    transforms.Compose([
        transforms.RandomRotation(degrees = 15),
        transforms.Resize(size=256),
        #transforms.ToTensor(),
    ]),
    
    'ColorJitter':
    transforms.Compose([
        transforms.ColorJitter(brightness = .1, contrast = .1, saturation = .1, hue = .05),
        transforms.Resize(size=256),
        #transforms.ToTensor(),
    ]),
    
    'RandomHorizontalFlip':
    transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize(size=256),
        #transforms.ToTensor(),
    ]),
}

random_resized_crop = transform['RandomResizedCrop']
random_crop = transform['RandomCrop']
random_rotation = transform['RandomRotation']
color_jitter = transform['ColorJitter']
random_horizontal_flip = transform['RandomHorizontalFlip']

Para poder guardar las imagenes aumentadas en la carpeta es necesario crear las subcarpetas de las clases (1 a 5) para poder asociar cada imagen a la clase correcta al cargarlas. 

In [32]:

import matplotlib.image as mpimg
import cv2
path = "train/"


images_class = []
images_classes = []

n_images = 0
number_images_augment = 0
images_augmented = 0
new_img = None

for i in range(1,6):
    
    print("---------------------------")
    print("class ", i)
    print("----------------------------")
    class_path = path + str(i) + "/"
    img_list = os.listdir(class_path)
    number_images_augment = max_images - n_images_class[i - 1]
    print("Augmenting with ", number_images_augment, " images for class ", i)
    images_augmented = 0
    
    for images_augmented in tqdm(range(number_images_augment)):
        r = random.randint(0, len(img_list) - 1) # Augment a random image
        
        img = Image.open(path + str(i) + "/" + img_list[r]).convert("RGB")
        r2 = random.randint(0, 4) # select randomly the transformation for the augmented img
            
        if r2 == 0:
            new_img = random_resized_crop(img)
        elif r2 == 1:
            new_img = random_crop(img)
        elif r2 == 2:
            new_img = random_rotation(img)
        elif r2 == 3:
            new_img = color_jitter(img)
        elif r2 == 4:
            new_img = random_horizontal_flip(img)
                
        new_img.save(path + str(i) + "/" + "_augmented_" + str(images_augmented) + ".jpg")
            
            
        new_img = None
        images_augmented += 1
    print("images augmented: ", images_augmented)

---------------------------
class  1
----------------------------
Augmenting with  0  images for class  1


0it [00:00, ?it/s]

images augmented:  0
---------------------------
class  2
----------------------------
Augmenting with  6  images for class  2


  0%|          | 0/6 [00:00<?, ?it/s]

images augmented:  6
---------------------------
class  3
----------------------------
Augmenting with  863  images for class  3


...

images augmented:  863
---------------------------
class  4
----------------------------
Augmenting with  2  images for class  4


  0%|          | 0/2 [00:00<?, ?it/s]

images augmented:  2
---------------------------
class  5
----------------------------
Augmenting with  0  images for class  5


0it [00:00, ?it/s]

images augmented:  0
